# Tutorial 3 - An advanced pipeline with multichannel signals.

We can use different channels for the receiver  to handle specifically each channel 
name. This can be useful for example in splitting train and test channels in machine 
learning Then, the user will need to implement specific handling of each channel in 
the receiving agent.
 
In this example, the *MultiGeneratorAgent* is used to send two different types of 
data - Sine and Cosine generator. This is done via specifying `send_output
(channel="sine")` and `send_output(channel="cosine")`.

Then on the receiving end, the `on_received_message()` function checks for 
`message['channel']` to handle it separately.

Note that by default, *MonitorAgent* is only subscribed to the `"default"` channel. 
Hence, it will not respond to the `"cosine"` and `"sine"` channel.

In [2]:
# %load tutorial_3_multi_channel.py
from agentMET4FOF.agents import AgentMET4FOF, AgentNetwork, MonitorAgent
from agentMET4FOF.streams import SineGenerator, CosineGenerator


class MultiGeneratorAgent(AgentMET4FOF):

    _sine_stream: SineGenerator
    _cos_stream: CosineGenerator

    def init_parameters(self):
        self._sine_stream = SineGenerator()
        self._cos_stream = CosineGenerator()

    def agent_loop(self):
        if self.current_state == "Running":
            sine_data = self._sine_stream.next_sample()  # dictionary
            cosine_data = self._cos_stream.next_sample()  # dictionary
            self.send_output(sine_data["quantities"], channel="sine")
            self.send_output(cosine_data["quantities"], channel="cosine")


class MultiOutputMathAgent(AgentMET4FOF):

    _minus_param: float
    _plus_param: float

    def init_parameters(self, minus_param=0.5, plus_param=0.5):
        self._minus_param = minus_param
        self._plus_param = plus_param

    def on_received_message(self, message):
        """
        Checks for message['channel'] and handles them separately
        Acceptable channels are "cosine" and "sine"
        """
        if message["channel"] == "cosine":
            minus_data = self.minus(message["data"], self._minus_param)
            self.send_output({"cosine_minus": minus_data})
        elif message["channel"] == "sine":
            plus_data = self.plus(message["data"], self._plus_param)
            self.send_output({"sine_plus": plus_data})

    @staticmethod
    def minus(data, minus_val):
        return data - minus_val

    @staticmethod
    def plus(data, plus_val):
        return data + plus_val


def main():
    # start agent network server
    agentNetwork = AgentNetwork()

    # init agents
    gen_agent = agentNetwork.add_agent(agentType=MultiGeneratorAgent)
    multi_math_agent = agentNetwork.add_agent(agentType=MultiOutputMathAgent)
    monitor_agent = agentNetwork.add_agent(agentType=MonitorAgent)

    # connect agents : We can connect multiple agents to any particular agent
    # However the agent needs to implement handling multiple inputs
    agentNetwork.bind_agents(gen_agent, multi_math_agent, channel=["sine", "cosine"])
    agentNetwork.bind_agents(multi_math_agent, monitor_agent)

    # set all agents states to "Running"
    agentNetwork.set_running_state()

    # allow for shutting down the network after execution
    return agentNetwork


if __name__ == "__main__":
    main()

Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333
INFO [2021-07-01 15:01:52.175496] (MultiGeneratorAgent_1): INITIALIZED
INFO [2021-07-01 15:01:52.210891] (MultiOutputMathAgent_1): INITIALIZED
INFO [2021-07-01 15:01:52.249834] (MonitorAgent_1): INITIALIZED
[2021-07-01 15:01:52.267296] (MultiGeneratorAgent_1): Connected output module: MultiOutputMathAgent_1
[2021-07-01 15:01:52.281811] (MultiOutputMathAgent_1): Connected output module: MonitorAgent_1

--------------------------------------------------------------
|                                                            |
| Your agent network is starting up. Open your browser and   |
| visit the agentMET4FOF dashboard on http://127.0.0.1:8050/ |
|                                                            |
--------------------------------------------------------------

SET STATE:   Running
[2021-07-01 15:01:53.182613] (MultiGeneratorAgen

[2021-07-01 15:01:56.189350] (MultiOutputMathAgent_1): Tproc: 0.00465
[2021-07-01 15:01:56.202432] (MonitorAgent_1): Buffer: {'MultiOutputMathAgent_1': {'sine_plus': array([0.5       , 1.08778525, 1.45105652, 1.45105652]), 'cosine_minus': array([0.5       , 0.49802673, 0.4921147 , 0.48228725])}}
[2021-07-01 15:01:56.192299] (MultiOutputMathAgent_1): Received: {'from': 'MultiGeneratorAgent_1', 'data': array([0.98228725]), 'senderType': 'MultiGeneratorAgent', 'channel': 'cosine'}
[2021-07-01 15:01:56.203192] (MonitorAgent_1): Tproc: 0.005025
[2021-07-01 15:01:56.194134] (MultiOutputMathAgent_1): Pack time: 0.001021
[2021-07-01 15:01:56.196129] (MultiOutputMathAgent_1): Sending: {'cosine_minus': array([0.48228725])}
[2021-07-01 15:01:56.196786] (MultiOutputMathAgent_1): Tproc: 0.003774
[2021-07-01 15:01:57.181941] (MultiGeneratorAgent_1): Pack time: 0.001002
[2021-07-01 15:01:57.187711] (MultiOutputMathAgent_1): Received: {'from': 'MultiGeneratorAgent_1', 'data': array([0.58778525]), 'sen

[2021-07-01 15:02:00.190476] (MultiOutputMathAgent_1): Pack time: 0.000718
[2021-07-01 15:02:00.229208] (MonitorAgent_1): Buffer: {'MultiOutputMathAgent_1': {'sine_plus': array([ 0.5       ,  1.08778525,  1.45105652,  1.45105652,  1.08778525,
        0.5       , -0.08778525, -0.45105652]), 'cosine_minus': array([0.5       , 0.49802673, 0.4921147 , 0.48228725, 0.46858316,
       0.45105652, 0.42977649])}}
[2021-07-01 15:02:00.229840] (MonitorAgent_1): Tproc: 0.021469
[2021-07-01 15:02:00.196944] (MultiOutputMathAgent_1): Sending: {'sine_plus': array([-0.45105652])}
[2021-07-01 15:02:00.232820] (MonitorAgent_1): Received: {'from': 'MultiOutputMathAgent_1', 'data': {'cosine_minus': array([0.40482705])}, 'senderType': 'MultiOutputMathAgent', 'channel': 'default'}
[2021-07-01 15:02:00.201511] (MultiOutputMathAgent_1): Tproc: 0.011616
[2021-07-01 15:02:00.241196] (MonitorAgent_1): Buffer: {'MultiOutputMathAgent_1': {'sine_plus': array([ 0.5       ,  1.08778525,  1.45105652,  1.45105652,  1.0